# Basic Strategy for Dealing with Small Datasets¶

* Create a naive convnet model without regularization to set a baseline for what can be achieved.
* Data Augmentation: to prevent overfitting.
* Feature extraction with a pretrained network.
* Fine tuning a pretrained network.

In this notebook, we augment our training samples to prevent overfitting.